In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, regularizers

import numpy as np


X_files = [
    "final_data_X.npy"
]

Y_files = [

    "final_data_Y.npy"
]


X = np.concatenate([np.load(file) for file in X_files], axis=0)
Y = np.concatenate([np.load(file) for file in Y_files], axis=0)

# Check the shapes of the combined arrays
print("Combined X shape:", X.shape)
print("Combined Y shape:", Y.shape)

Combined X shape: (872950, 6, 7, 2)
Combined Y shape: (872950,)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from collections import defaultdict

def consolidate_duplicates(X, Y):
    move_counts = defaultdict(lambda: defaultdict(int))

    # Count occurrences of Y for each unique X
    for x, y in zip(X, Y):
        x_key = tuple(map(tuple, x.reshape(-1, x.shape[-1])))  # Convert to hashable key
        move_counts[x_key][y] += 1

    # Select the most frequent Y for each X
    consolidated_data = []
    for board_key, moves in move_counts.items():
        most_frequent_move = max(moves, key=moves.get)
        consolidated_data.append((np.array(board_key).reshape(X.shape[1:]), most_frequent_move))

    # Convert to numpy arrays
    consolidated_X = np.array([item[0] for item in consolidated_data])
    consolidated_Y = np.array([item[1] for item in consolidated_data])
    return consolidated_X, consolidated_Y

# Apply the function to the current X and Y
consolidated_X, consolidated_Y = consolidate_duplicates(X, Y)

# Check the shapes of the combined arrays
print("Combined X shape:", consolidated_X.shape)
print("Combined Y shape:", consolidated_Y.shape)

Combined X shape: (660642, 6, 7, 2)
Combined Y shape: (660642,)


In [ ]:
# np.save('final_X.npy', consolidated_X)
# np.save('final_Y.npy', consolidated_Y)

In [ ]:

# def combine_with_mirrored_dataset(X, Y):
#     mirrored_X = []
#     mirrored_Y = []

#     for x, y in zip(X, Y):
#         # Create the mirror image of the board
#         mirrored_board = np.flip(x, axis=1)  # Flip horizontally (mirror image)
#         mirrored_X.append(mirrored_board)

#         # Update the column index for the mirrored board
#         mirrored_column = X.shape[2] - 1 - y  # New column index after mirroring
#         mirrored_Y.append(mirrored_column)

#     # Convert mirrored data to numpy arrays
#     mirrored_X = np.array(mirrored_X)
#     mirrored_Y = np.array(mirrored_Y)

#     # Combine original and mirrored data
#     combined_X = np.concatenate((X, mirrored_X), axis=0)
#     combined_Y = np.concatenate((Y, mirrored_Y), axis=0)

#     return combined_X, combined_Y

# # Apply the function to combine the original and mirrored dataset
# combined_X, combined_Y = combine_with_mirrored_dataset(consolidated_X, consolidated_Y)

# # Check the shapes of the combined dataset
# print("Combined X shape:", combined_X.shape)
# print("Combined Y shape:", combined_Y.shape)

# # Save the combined dataset if needed
# np.save('combined_X.npy', combined_X)
# np.save('combined_Y.npy', combined_Y)


Combined X shape: (872950, 6, 7, 2)
Combined Y shape: (872950,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


# Ensure all moves are for 'plus' player by flipping boards if necessary
def preprocess_board(board, move, player):
    if player == 'minus':
        board = np.flip(board, axis=-1)  # Swap the last dimension
        move = 6 - move  # Mirror the move for the column
    return board, move

# Preprocess the dataset
# X_processed = []
# Y_processed = []
# for board, move in zip(combined_X, combined_Y):
#     board, move = preprocess_board(board, move, 'plus')  # Convert to 'plus'
#     X_processed.append(board)
#     Y_processed.append(move)

# X_processed = np.array(X_processed)
# Y_processed = np.array(Y_processed)

# Split into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(consolidated_X, consolidated_Y, test_size=0.2, random_state=42)

# Normalize inputs (optional, since values are binary)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# One-hot encode the target variable
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=7)
Y_val = tf.keras.utils.to_categorical(Y_val, num_classes=7)

print('X_train Shape:', X_train.shape)

# CNN Model
def create_cnn():
  model = Sequential([
          # Capture horizontal, vertical, and diagonal patterns
          Conv2D(128, kernel_size=(4,4), activation='relu', input_shape=(6, 7, 2), padding='same'),
          BatchNormalization(),

          Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
          BatchNormalization(),

          Conv2D(128, kernel_size=(2,2), activation='relu', padding='same'),
          MaxPooling2D(pool_size=(2,2)),

          # Additional layers to learn complex strategies
          Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'),
          BatchNormalization(),

          Flatten(),
          Dense(512, activation='relu'),
          Dropout(0.4),
          Dense(256, activation='relu'),
          Dense(7, activation='softmax')  # 7 possible moves (columns)
      ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

cnn_model = create_cnn()
cnn_model.summary()

# Learning rate scheduler and early stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Train CNN
cnn_history = cnn_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=50,
    batch_size=32,
    callbacks=[lr_scheduler, early_stop],
    verbose=1
)



X_train Shape: (528513, 6, 7, 2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 6, 7, 128)           │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 6, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 6, 7, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 6, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 6, 7, 128)           │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 3, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 3, 3, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,827,975 (6.97 MB)

 Trainable params: 1,826,951 (6.97 MB)

 Non-trainable params: 1,024 (4.00 KB)

Epoch 1/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.3901 - loss: 1.5464 - val_accuracy: 0.6114 - val_loss: 0.9836 - learning_rate: 0.0010
Epoch 2/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 141s 5ms/step - accuracy: 0.6205 - loss: 0.9690 - val_accuracy: 0.6483 - val_loss: 0.8779 - learning_rate: 0.0010
Epoch 3/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 133s 5ms/step - accuracy: 0.6494 - loss: 0.8793 - val_accuracy: 0.6620 - val_loss: 0.8496 - learning_rate: 0.0010
Epoch 4/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 76s 5ms/step - accuracy: 0.6673 - loss: 0.8329 - val_accuracy: 0.6601 - val_loss: 0.8407 - learning_rate: 0.0010
Epoch 5/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - accuracy: 0.6773 - loss: 0.8062 - val_accuracy: 0.6682 - val_loss: 0.8472 - learning_rate: 0.0010
Epoch 6/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 81s 5ms/step - accuracy: 0.6868 - loss: 0.7816 - val_accuracy: 0.6723 - val_loss: 0.8074 - learning_rate: 0.0010
Epoch 7/50
16517/16517 ━━━━━━━━━━━━━━━━━━━━ 85s 5ms/step

In [ ]:
cnn_model.save('connect4_cnn_model.h5')

In [ ]:
from google.colab import files
files.download('connect4_cnn_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>